<a href="https://colab.research.google.com/github/BestQuark/etech/blob/main/encrypt_face_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y --fix-missing \
    build-essential \
    cmake \
    gfortran \
    git \
    wget \
    curl \
    graphicsmagick \
    libgraphicsmagick1-dev \
    libatlas-base-dev \
    libavcodec-dev \
    libavformat-dev \
    libgtk2.0-dev \
    libjpeg-dev \
    liblapack-dev \
    libswscale-dev \
    pkg-config \
    python3-dev \
    python3-numpy \
    software-properties-common \
    zip \
    && apt-get clean && rm -rf /tmp/* /var/tmp/*

!cd ~ && \
    mkdir -p dlib && \
    git clone -b 'v19.9' --single-branch https://github.com/davisking/dlib.git dlib/ && \
    cd  dlib/ && \
    python3 setup.py install --yes USE_AVX_INSTRUCTIONS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
!git clone https://github.com/davisking/dlib.git
!cd dlib
!mkdir build; cd build; cmake ..; cmake --build .
!cd ..
!python3 setup.py install
!pip install cmake
!pip install face_recognition cryptography
!pip install pypng
!pip install pyzbar
!sudo apt-get install libzbar0
!pip install face_recognition pyqrcode

     |████████████████████████████████| 3.6 MB 5.3 MB/s 


In [169]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cryptography.fernet import Fernet
import pyqrcode
import face_recognition
from PIL import Image
from pyzbar.pyzbar import decode

In [272]:
robustness = 3
def gen_qrcode(pic_path, public_key):
    pic = face_recognition.load_image_file(pic_path)
    pic_encoding = face_recognition.face_encodings(pic)[0]
    low_res_enc = np.around(pic_encoding,robustness)
    f = Fernet(public_key)
    token = f.encrypt(str.encode(''.join(str(e) for e in low_res_enc)))
    code = pyqrcode.create(token)
    #code.svg('code.svg', scale=4, background="white", module_color="#7D007D")
    code.png('code.png', scale=6)

def decode_qr(qr_code_path):
    qcode = cv2.imread(qr_code_path)	
    return decode(qcode)[0][0]

def decode_encrypted(encod_str, public_key):
    f = Fernet(public_key)
    return f.decrypt(encod_str)

def check_match(qr_face, current_image_path):
    pic = face_recognition.load_image_file(current_image_path)
    pic_encoding = face_recognition.face_encodings(pic)[0]
    st = qr_face.decode()
    k = ''
    t=False
    for e in st:
        if e=='-':
            k+=';'
            t=True
        elif e=='.':
            if t==True: t=False
            else: k+=';'
        k+=e
    if k[0]==';':k=k[1:]
    if k[-1]==';':k=k[:-1]
    qr_face_encoding = np.fromstring(k, sep=';')
    return face_recognition.compare_faces([qr_face_encoding],pic_encoding, tolerance=0.5)

In [278]:
key = Fernet.generate_key()

In [273]:
gen_qrcode("pic.jpg", key)

In [275]:
qrscan = decode_qr("code.png")

In [276]:
pl = decode_encrypted(qrscan, key)

In [277]:
check_match(pl, "pic2.jpg")

[True]